In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import warnings
warnings.filterwarnings('ignore')

## Import & Prepare DataFrame

In [2]:
def remove_whitespaces(df: pd.DataFrame) -> None:
    """Remove whitespaces from column names and string values

    Parameters
    ----------
    df : pd.DataFrame
        Dataframe to clean
    """
    # Remove whitespace from each column name
    df.columns = df.columns.str.strip()
    
    # Remove whitespace from each string value
    categorical_columns = df.select_dtypes("O").columns
    for column in categorical_columns:
        df[column] = df[column].str.strip()

In [3]:
MATCHES_DATA_PATH = "../data/matches_data/historical_matches.csv"
PLAYERS_DATA_PATH = "../data/players_data/players_all_prepared.csv"

# Load the matches data
matches = pd.read_csv(MATCHES_DATA_PATH)

# Load the players data
players = pd.read_csv(PLAYERS_DATA_PATH)


# Remove whitespaces from column names and string values
remove_whitespaces(matches)
remove_whitespaces(players)

In [4]:
# Matches date to datetime
matches["date"] = pd.to_datetime(matches["date"])
matches["year"] = matches["date"].dt.year

# Filter years in matches to match the given players data dates [2015, 2022]
matches = matches[matches["year"] >= 2014]

# Filter tournaments to keep only ones  having players as in world cup matches
to_keep_tournaments = ["Friendly", 
"AFC Asian Cup qualification",
"EAFF Championship",
"African Cup of Nations qualification",
"FIFA World Cup",
"Kirin Challenge Cup",
"UEFA Euro qualification",
"Superclásico de las Américas",
"Gulf Cup",
"AFC Asian Cup",
"African Cup of Nations",
"FIFA World Cup qualification",
"Copa América",
"Gold Cup",
"Copa América qualification",
"Kirin Cup",
"UEFA Euro",
"Confederations Cup",
"UEFA Nations League",
"CONCACAF Nations League qualification",
"CONCACAF Nations League",
"CONMEBOL–UEFA Cup of Champions"]

In [5]:
## Assert all to_keep torunments are in dataframe
for torn in to_keep_tournaments:
    all_tournaments = matches["tournament"].unique()
    found = matches[matches["tournament"] == torn].count().sum()
    assert found > 0, f"Zero mathces found for tournament {torn}"
    assert (all_tournaments == torn).sum() == 1, f"tournament {torn} NOT found in matches"


In [6]:
# Filter matches to keep only the ones in the tournaments to keep
matches = matches[matches["tournament"].isin(to_keep_tournaments)]

# matches net score
matches["net_score"] = matches["home_score"] - matches["away_score"]

# Keep only desired columns
matches = matches[["year", "home_team", "away_team", "net_score"]]

# Rename columns
matches.rename(columns={"home_team": "1st_team", "away_team": "2nd_team"}, inplace=True)

In [7]:
matches.reset_index(drop=True, inplace=True)
players.reset_index(drop=True, inplace=True)

## Merge Data

In [8]:
## Adjust year value. The FIFA data collected in 2023 represents the year 2022, and this applies to all the data.
## This adjusment is to correctly merge matches with the corresponding players data 
players["year"] -= 1


In [17]:
## get unique positions
all_positions = players["nation_position"].unique()
all_positions

array(['GK', 'RWB', 'LCB', 'SUB', 'LCM', 'RW', 'LW', 'RCM', 'LWB', 'RCB',
       'CB', 'ST', 'RES', 'LB', 'RDM', 'LM', 'RB', 'RM', 'LDM', 'CDM',
       'RS', 'CAM', 'LS', 'LF', 'RF', 'CF', 'CM', 'LAM', 'RAM'],
      dtype=object)

In [10]:
drop_ind = []
for i, row in matches.iterrows():
    year = row["year"]
    team1 = row["1st_team"].lower().strip()
    team2 = row["2nd_team"].lower().strip()

    # Check if team1 is in players data
    team1_players = players[(players["year"] == year) & (players["nationality_name"].str.lower() == team1)]
    if team1_players.empty:
        drop_ind.append(i)
        continue

    # Check if team2 is in players data 
    team2_players = players[(players["year"] == year) & (players["nationality_name"].str.lower()== team2)]
    if team2_players.empty:
        drop_ind.append(i)
        continue

In [11]:
# invalid matches ratio
len(drop_ind)/matches.shape[0]

0.8362068965517241

In [12]:
matches.drop(drop_ind, inplace=True)

In [13]:
grouped_players = players.groupby(["nationality_name", "year", "nation_position"]).mean()

In [22]:
players[(players["nationality_name"] == "Poland") & (players["year"]==2014)]["nation_position"].unique()

array(['LS', 'SUB', 'GK', 'LCB', 'RCM', 'RM', 'LM', 'LCM', 'RS', 'LB',
       'RCB', 'RB'], dtype=object)

In [39]:
grouped_players.columns.shape[0]

38

In [41]:
np.zeros((1,grouped_players.columns.shape[0]))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])

In [42]:
plyrs_record2 = pd.DataFrame(np.zeros((1,grouped_players.columns.shape[0])), columns=grouped_players.columns)
plyrs_record2

,height_cm,weight_kg,age,overall,potential,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,...,mentality_vision,mentality_penalties,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
new_row

,year,1st_team,2nd_team,net_score,RAM_height_cm_2nd,RAM_weight_kg_2nd,RAM_age_2nd,RAM_overall_2nd,RAM_potential_2nd,RAM_attacking_crossing_2nd,...,RAM_mentality_vision_2nd,RAM_mentality_penalties_2nd,RAM_defending_marking_awareness_2nd,RAM_defending_standing_tackle_2nd,RAM_defending_sliding_tackle_2nd,RAM_goalkeeping_diving_2nd,RAM_goalkeeping_handling_2nd,RAM_goalkeeping_kicking_2nd,RAM_goalkeeping_positioning_2nd,RAM_goalkeeping_reflexes_2nd
0,2014,Norway,Poland,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
dataset

,year,1st_team,2nd_team,net_score,GK_height_cm_2nd,GK_weight_kg_2nd,GK_age_2nd,GK_overall_2nd,GK_potential_2nd,GK_attacking_crossing_2nd,...,RAM_mentality_vision_2nd,RAM_mentality_penalties_2nd,RAM_defending_marking_awareness_2nd,RAM_defending_standing_tackle_2nd,RAM_defending_sliding_tackle_2nd,RAM_goalkeeping_diving_2nd,RAM_goalkeeping_handling_2nd,RAM_goalkeeping_kicking_2nd,RAM_goalkeeping_positioning_2nd,RAM_goalkeeping_reflexes_2nd
0,2014,Norway,Poland,-3.0,196.0,84.0,24.0,80.0,83.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
pd.DataFrame(match).T

,year,1st_team,2nd_team,net_score
1,2014,Australia,Ecuador,-1.0


In [97]:
dataset = matches.copy().reset_index(drop=True)
error_count = 0
counter = 0
for i, match in dataset.iterrows():
    for pos in all_positions:
        # new_row = pd.DataFrame()
        second_team =  match["2nd_team"]
        first_team = match["1st_team"]
        year = match["year"]
        counter+=1
        try:
            # Get players data for first team
            plyrs_record1 = pd.DataFrame(grouped_players.loc[first_team, year, pos])
            plyrs_record2 = pd.DataFrame(grouped_players.loc[second_team, year, pos])
            plyrs_record1 = plyrs_record1.T.reset_index(drop=True)
            plyrs_record2 = plyrs_record2.T.reset_index(drop=True)
        except:
            plyrs_record2 = pd.DataFrame(np.zeros((1,grouped_players.columns.shape[0])), columns=grouped_players.columns)
            plyrs_record1 = pd.DataFrame(np.zeros((1,grouped_players.columns.shape[0])), columns=grouped_players.columns)

        
        
        plyrs_record2.columns = pos + "_" + plyrs_record2.columns + "_2nd"
        plyrs_record1.columns = pos + "_" + plyrs_record1.columns + "_1st"
        dataset_cols = dataset.columns
        dataset = pd.concat([pd.DataFrame(dataset.loc[i]).T, plyrs_record2], axis=1, ignore_index=True)
        dataset = pd.concat([pd.DataFrame(dataset.loc[i]).T, plyrs_record1], axis=1, ignore_index=True)
        dataset.columns = dataset_cols.tolist() + plyrs_record2.columns.tolist() + plyrs_record1.columns.tolist()
        # print(new_row)
    # dataset = pd.concat([dataset, new_row], axis=0, ignore_index=True)
        # break
    break
    # dataset = pd.concat([dataset, pd.DataFrame(match).T], axis=0, ignore_index=True)
    
        
        
        
    

In [98]:
dataset

,year,1st_team,2nd_team,net_score,GK_height_cm_2nd,GK_weight_kg_2nd,GK_age_2nd,GK_overall_2nd,GK_potential_2nd,GK_attacking_crossing_2nd,...,RAM_mentality_vision_1st,RAM_mentality_penalties_1st,RAM_defending_marking_awareness_1st,RAM_defending_standing_tackle_1st,RAM_defending_sliding_tackle_1st,RAM_goalkeeping_diving_1st,RAM_goalkeeping_handling_1st,RAM_goalkeeping_kicking_1st,RAM_goalkeeping_positioning_1st,RAM_goalkeeping_reflexes_1st
0,2014,Norway,Poland,-3.0,196.0,84.0,24.0,80.0,83.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
